In [2]:
import h5py
import numpy as np
from numpy import newaxis
import cv2
import sys
import os
from image_utils import read_data_h5,write_data_h5
import matplotlib.pyplot as plt

In [3]:
if __name__ == '__main__':
    

    path_patches='../DATA_GHANA/RAW_PATCHES/360_x_360/'
    path_dataset='../DATA_GHANA/DATASET/360_x_360_8_bands/'
    if not os.path.exists(path_dataset):
            os.makedirs(path_dataset)
    
   
    training_ratio=0.8 #so    test_ratio=0.2
    validation_ratio=0.2
    
    
    list_input_panchro=[]
    list_input_ms=[]
    list_input_pansharp=[]
    for filename in sorted(os.listdir(path_patches)):
        if filename.startswith('panchro'):
            print('Reading %s'%filename)
            list_input_panchro.append(read_data_h5(path_patches+filename))
#         if filename.startswith('ms'):
#             print('Reading %s'%filename)
#             list_input_ms.append(read_data_h5(path_patches+filename))
        if filename.startswith('pansharp'):
            print('Reading %s'%filename)
            list_input_pansharp.append(read_data_h5(path_patches+filename))
        if filename.startswith('groundtruth'):
            print('Reading %s'%filename)
            list_output=read_data_h5(path_patches+filename)
            
#     ##BUILD Panchro + Pansharp --> 9 bands
    
    list_input_panchro=np.squeeze(np.asarray(list_input_panchro))[newaxis,:,:,:]
    list_input_pansharp=np.squeeze(np.asarray(list_input_pansharp))
    list_input=np.concatenate((list_input_panchro,list_input_pansharp),axis=0)
    
#     ## Build Pancrho + MS --> 9 bands
    
#     list_input_panchro=np.squeeze(np.asarray(list_input_panchro))[newaxis,:,:,:]
#     list_input_ms=np.squeeze(np.asarray(list_input_ms))
#     list_input=np.concatenate((list_input_panchro,list_input_ms),axis=0)
    
#     ## Build Panchro + Pansharp 2,3,5,7

#     list_input_panchro=np.squeeze(np.asarray(list_input_panchro))[newaxis,:,:,:]
#     list_input_pansharp=np.stack((np.squeeze(np.asarray(list_input_pansharp))[1,:,:,:],
#                                         np.squeeze(np.asarray(list_input_pansharp))[2,:,:,:],
#                                         np.squeeze(np.asarray(list_input_pansharp))[4,:,:,:],
#                                         np.squeeze(np.asarray(list_input_pansharp))[6,:,:,:]),axis=0)
#     list_input=np.concatenate((list_input_panchro,list_input_pansharp),axis=0)

    ## Build Panchro + Pansharp 2,3,5,7 + MS -->14 bands
    
#     list_input_panchro=np.squeeze(np.asarray(list_input_panchro))[newaxis,:,:,:]
#     list_input_pansharp=np.stack((np.squeeze(np.asarray(list_input_pansharp))[1,:,:,:],
#                                         np.squeeze(np.asarray(list_input_pansharp))[2,:,:,:],
#                                         np.squeeze(np.asarray(list_input_pansharp))[4,:,:,:],
#                                         np.squeeze(np.asarray(list_input_pansharp))[6,:,:,:]),axis=0)
#     list_input_ms=np.squeeze(np.asarray(list_input_ms))
#     list_input=np.concatenate((list_input_panchro,list_input_pansharp,list_input_ms),axis=0)
    
    
    
    ## Followup
    list_input=np.transpose(list_input,(1,2,3,0))
    
    list_output=np.squeeze(list_output)
    
    print('Dataset read')
    idx_shuffle = np.arange(len(list_input))
    np.random.shuffle(idx_shuffle)
    print('Dataset shuffled')    
    list_input=list_input[idx_shuffle]
    list_output=list_output[idx_shuffle]
    
    #Do the split
    training_size=int(round(training_ratio*list_input.shape[0]))
    test_size=list_input.shape[0]-training_size
    validation_size=int(round(validation_ratio*training_size))
    training_size=training_size-validation_size
    
    
    print('Split (TRAINING - VALIDATION:%f) - TEST:%f  done'%(1-validation_ratio,training_ratio))
    print('Training size:%d, Validation size:%d, Test size: %d'%(training_size,validation_size,test_size))
    
    #Save the dataset
    
    if not os.path.exists(path_dataset+'TRAINING'):
            os.makedirs(path_dataset+'TRAINING')
            if not os.path.exists(path_dataset+'TRAINING/INPUT'):
                os.makedirs(path_dataset+'TRAINING/INPUT')
            if not os.path.exists(path_dataset+'TRAINING/OUTPUT'):
                os.makedirs(path_dataset+'TRAINING/OUTPUT')
    if not os.path.exists(path_dataset+'VALIDATION'):
            os.makedirs(path_dataset+'VALIDATION')
            if not os.path.exists(path_dataset+'VALIDATION/INPUT'):
                os.makedirs(path_dataset+'VALIDATION/INPUT')
            if not os.path.exists(path_dataset+'VALIDATION/OUTPUT'):
                os.makedirs(path_dataset+'VALIDATION/OUTPUT')
    if not os.path.exists(path_dataset+'TEST'):
            os.makedirs(path_dataset+'TEST')
            if not os.path.exists(path_dataset+'TEST/INPUT'):
                os.makedirs(path_dataset+'TEST/INPUT')
            if not os.path.exists(path_dataset+'TEST/OUTPUT'):
                os.makedirs(path_dataset+'TEST/OUTPUT')
            
    print('BUILD TRAINING SET')
    for i in range(training_size):
        print('Patch %d'%i)
        write_data_h5(path_dataset+'TRAINING/INPUT/input_'+str(i)+'.h5',list_input[i,:,:,:])
        write_data_h5(path_dataset+'TRAINING/OUTPUT/output_'+str(i)+'.h5',list_output[i,:,:])
        
    print('BUILD VALIDATION SET')
    for i in range(training_size,training_size+validation_size):
        print('Patch %d'%i)
        write_data_h5(path_dataset+'VALIDATION/INPUT/input_'+str(i)+'.h5',list_input[i,:,:,:])
        write_data_h5(path_dataset+'VALIDATION/OUTPUT/output_'+str(i)+'.h5',list_output[i,:,:])
        
    print('BUILD TEST SET')
    for i in range(training_size+validation_size,list_input.shape[0]):
        print('Patch %d'%i)
        write_data_h5(path_dataset+'TEST/INPUT/input_'+str(i)+'.h5',list_input[i,:,:,:])
        write_data_h5(path_dataset+'TEST/OUTPUT/output_'+str(i)+'.h5',list_output[i,:,:])
            
    
            
        

Reading groundtruth.h5
Reading panchro.h5
Reading pansharpened_0.h5
Reading pansharpened_1.h5
Reading pansharpened_2.h5
Reading pansharpened_3.h5
Reading pansharpened_4.h5
Reading pansharpened_5.h5
Reading pansharpened_6.h5
Reading pansharpened_7.h5
Dataset read
Dataset shuffled
Split (TRAINING - VALIDATION:0.800000) - TEST:0.800000  done
Training size:570, Validation size:143, Test size: 178
BUILD TRAINING SET
Patch 0
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_0.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_0.h5 created
Patch 1
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_1.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_1.h5 created
Patch 2
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_2.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_2.h5 created
Patch 3
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_3.h5 created
F

File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_46.h5 created
Patch 47
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_47.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_47.h5 created
Patch 48
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_48.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_48.h5 created
Patch 49
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_49.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_49.h5 created
Patch 50
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_50.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_50.h5 created
Patch 51
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_51.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_51.h5 created
Patch 52
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_

File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_96.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_96.h5 created
Patch 97
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_97.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_97.h5 created
Patch 98
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_98.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_98.h5 created
Patch 99
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_99.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_99.h5 created
Patch 100
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_100.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_100.h5 created
Patch 101
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_101.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_101.

File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_144.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_144.h5 created
Patch 145
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_145.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_145.h5 created
Patch 146
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_146.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_146.h5 created
Patch 147
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_147.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_147.h5 created
Patch 148
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_148.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_148.h5 created
Patch 149
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_149.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/

File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_192.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_192.h5 created
Patch 193
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_193.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_193.h5 created
Patch 194
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_194.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_194.h5 created
Patch 195
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_195.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_195.h5 created
Patch 196
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_196.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/output_196.h5 created
Patch 197
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/INPUT/input_197.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TRAINING/OUTPUT/

...

File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/OUTPUT/output_672.h5 created
Patch 673
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/INPUT/input_673.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/OUTPUT/output_673.h5 created
Patch 674
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/INPUT/input_674.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/OUTPUT/output_674.h5 created
Patch 675
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/INPUT/input_675.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/OUTPUT/output_675.h5 created
Patch 676
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/INPUT/input_676.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/OUTPUT/output_676.h5 created
Patch 677
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/INPUT/input_677.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/VALIDATION/OUTPUT/output_677.h5 created
Patch 678
File../DATA_GHANA/DATASET/

File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_719.h5 created
Patch 720
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_720.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_720.h5 created
Patch 721
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_721.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_721.h5 created
Patch 722
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_722.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_722.h5 created
Patch 723
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_723.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_723.h5 created
Patch 724
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_724.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_724.h5 created
Patch 725
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_725.h5 created
File../DATA_GHAN

File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_769.h5 created
Patch 770
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_770.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_770.h5 created
Patch 771
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_771.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_771.h5 created
Patch 772
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_772.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_772.h5 created
Patch 773
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_773.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_773.h5 created
Patch 774
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_774.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_774.h5 created
Patch 775
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_775.h5 created
File../DATA_GHAN

File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_819.h5 created
Patch 820
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_820.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_820.h5 created
Patch 821
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_821.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_821.h5 created
Patch 822
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_822.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_822.h5 created
Patch 823
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_823.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_823.h5 created
Patch 824
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_824.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_824.h5 created
Patch 825
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_825.h5 created
File../DATA_GHAN

File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_870.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_870.h5 created
Patch 871
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_871.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_871.h5 created
Patch 872
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_872.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_872.h5 created
Patch 873
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_873.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_873.h5 created
Patch 874
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_874.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_874.h5 created
Patch 875
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/INPUT/input_875.h5 created
File../DATA_GHANA/DATASET/360_x_360_8_bands/TEST/OUTPUT/output_875.h5 created
Patch 876
File../DATA_GHAN